In [1]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook

from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

2023-03-03 19:57:09.435425: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0


In [2]:
##GPU 사용 시
device = torch.device("cuda:0")
bertmodel, vocab = get_pytorch_kobert_model()

using cached model. /home/mglee/VSCODE/git_folder/comments_sentiment_label/code/.cache/kobert_v1.zip
using cached model. /home/mglee/VSCODE/git_folder/comments_sentiment_label/code/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


# 실험에 필요한 레이블들 준비

In [75]:
import pandas as pd
comments_data = pd.read_csv("../data/replaced_text.csv")

In [76]:
temp = comments_data.sample(frac=1).reset_index(drop=True)
temp['text'] = temp['text'].apply(lambda x : x.replace('\n', ' '))
temp['text'] = temp['text'].apply(lambda x : x.replace('\t', ' '))
temp['replaced text'] = temp['replaced text'].apply(lambda x : x.replace('\n' , ' '))
temp['replaced text'] = temp['replaced text'].apply(lambda x : x.replace('\t' , ' '))

In [77]:
train_test = temp

In [78]:
train_test['감정'] = temp['감정'].apply(lambda x: 1 if x == '긍정' else 0)

Kakao Pay 데이터를 unseen data로 성능 평가하기위한 데이터셋 준비

In [79]:
import pandas as pd
unseen_data = pd.read_excel("../../KakaoPay_ESG/data/comments_label/KP_senti.xlsx")

In [80]:
unseen_data.sample(frac=1).reset_index(drop=True)

,comments,sentiment
0,"""걍 메신저 없어져라단체톡 노예 하고싶지않다스트레스없는 평온한주말이 얼마만인지…""",0
1,카카오 없으니 불편하더만 ~ 슬슬 다른 커뮤니티로 갈아타야 할듯~,1
2,카카오가 얼마나 확장적이고 다각적으로 성장하고 있는데 우물안 개구리라니요,1
3,삼성전자가 카카오가 필요한 순간이 곧 오겠지,1
4,카카오톡 떠납니다. 애초에 카카오 관련 어플 못미더웠는데 잘 털고 가는듯,0
...,...,...
1662,돈에 대한 아이디어만 있어서 가장 악날한 방법으로 가장싸게 서민들 영역까지 침범하든...,0
1663,지금 파세요.,0
1664,얘넨 비상발전기도 없냐,0
1665,카카오가 탐욕의 아이콘이 되지 않길 바랍니다,0


In [81]:
unseen_target = [
    '카카오페이', '카카오뱅크','카카오', '개카오', '좌카오', '카페', '갓카오', '카뱅'
]

unseen_contrast = [
    '네이버', '토스', '정부', '소상공인', '서민', '개미'
]

In [82]:
replaced_text = []
for _, row in unseen_data.iterrows():
    txt = row['comments']
    for keyword in unseen_target:
        txt = txt.replace(keyword, "타겟")
    for keyword in unseen_contrast:
        txt = txt.replace(keyword, "대조")
    replaced_text.append(txt)

In [83]:
unseen_data['replaced text'] = replaced_text

In [84]:
unseen = unseen_data.sample(frac=1).reset_index(drop=True)
unseen['comments'] = unseen['comments'].apply(lambda x : x.replace('\n', ' '))
unseen['comments'] = unseen['comments'].apply(lambda x : x.replace('\t', ' '))
unseen['replaced text'] = unseen['replaced text'].apply(lambda x : x.replace('\n' , ' '))
unseen['replaced text'] = unseen['replaced text'].apply(lambda x : x.replace('\t' , ' '))

In [85]:
unseen.to_csv('../data/KP_unseen.tsv', index=False, header=None, sep = "\t")

In [86]:
unseen

,comments,sentiment,replaced text
0,컴퓨터공학과가 짱이구나,1,컴퓨터공학과가 짱이구나
1,아직도 복구못햇냐 전국민이 사용하는데 무능하다 카카오,0,아직도 복구못햇냐 전국민이 사용하는데 무능하다 타겟
2,대우가 생각난다. 임원ㅣㄴ들 미리 출국금지시켜놓던가 사람붙여 감시해라.,0,대우가 생각난다. 임원ㅣㄴ들 미리 출국금지시켜놓던가 사람붙여 감시해라.
3,카페가 우리나라 주식시장중 적자바이오기업빼고 제일거품,0,타겟가 우리나라 주식시장중 적자바이오기업빼고 제일거품
4,라이언 갈수록 보기 싫음. 카카오가 국민들 돈뜯어 먹는 깡패짓하는데 압잡이,0,라이언 갈수록 보기 싫음. 타겟가 국민들 돈뜯어 먹는 깡패짓하는데 압잡이
...,...,...,...
1662,양아치들 둘이 만나서 잘하는짓이네. 통신사는 불법보조금 풀어서 폰팔이들 앞장세우고 ...,0,양아치들 둘이 만나서 잘하는짓이네. 통신사는 불법보조금 풀어서 폰팔이들 앞장세우고 ...
1663,카카오가 은행 눌렀듯이 카카오도 다른 서비스에 의해 눌릴듯.... CS 잘 해야할듯,0,타겟가 은행 눌렀듯이 타겟도 다른 서비스에 의해 눌릴듯.... CS 잘 해야할듯
1664,카카오페이 정말 대단한 것 같습니다. 평소에 카카오페이를 많이 사용하지 않지만 주변...,1,타겟 정말 대단한 것 같습니다. 평소에 타겟를 많이 사용하지 않지만 주변 지인들의 ...
1665,"카카오페이, 사용자 맞춤형 ‘카드추천’ 서비스 시작, 기사 잘 보았습니다. 여러분!...",1,"타겟, 사용자 맞춤형 ‘카드추천’ 서비스 시작, 기사 잘 보았습니다. 여러분! 더치..."


In [87]:
comments_train = train_test[:1300]
comments_test = train_test[1300:]
comments_unseen = unseen

print(comments_train.shape[0])
print(comments_test.shape[0])
print(comments_unseen.shape[0])

1300
200
1667


In [19]:
comments_train.to_csv('../data/comments_train.tsv', index=False, header=True, sep = "\t")
comments_test.to_csv('../data/comments_test.tsv', index=False, header=True, sep = "\t")
comments_unseen.to_csv('../data/comments_unseen.tsv', index=False, header=True, sep = "\t")

# 한번만 선언하면 되는 애들

In [19]:
import pandas as pd
comments_unseen = pd.read_csv('../data/comments_unseen.tsv', sep = "\t")

In [3]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))
 
    def __len__(self):
        return (len(self.labels))


In [4]:
## Setting parameters
max_len = 64
batch_size = 32
warmup_ratio = 0.1
num_epochs = 5
max_grad_norm = 1
log_interval = 500
learning_rate =  5e-5

In [5]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=2,
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [6]:
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

# 네이버 영화리뷰로 학습

In [246]:
# !wget https://www.dropbox.com/s/374ftkec978br3d/ratings_train.txt?dl=1
# !wget https://www.dropbox.com/s/977gbwh542gdy94/ratings_test.txt?dl=1

In [74]:
dataset_train = nlp.data.TSVDataset("ratings_train.txt?dl=1", field_indices=[1,2], num_discard_samples=1)
dataset_test = nlp.data.TSVDataset("ratings_test.txt?dl=1", field_indices=[1,2], num_discard_samples=1)
dataset_unseen = nlp.data.TSVDataset("../data/comments_unseen.tsv", field_indices=[0,1], num_discard_samples=1)
print(dataset_train[0], '\n', dataset_test[0], '\n', dataset_unseen[0])

['아 더빙.. 진짜 짜증나네요 목소리', '0'] 
 ['굳 ㅋ', '1'] 
 ['공산당이야 뭐야 중국처럼되고싶어?', '0']


In [75]:
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

using cached model. /home/mglee/VSCODE/git_folder/comments_sentiment_label/code/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [76]:
data_train = BERTDataset(dataset_train, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok, max_len, True, False)
data_unseen = BERTDataset(dataset_unseen, 0, 1, tok, max_len, True, False)

train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)
unseen_dataloader = torch.utils.data.DataLoader(data_unseen, batch_size=batch_size, num_workers=5)

for data in train_dataloader:
  print("Data: ", data)
  print("Waveform: {}\nSample rate: {}\nLabels: {}".format(data[0], data[1], data[2]))
  break

Data:  [tensor([[   2, 3093, 1698,  ...,    1,    1,    1],
        [   2,  517, 7989,  ...,    1,    1,    1],
        [   2, 1458, 7191,  ...,    1,    1,    1],
        ...,
        [   2,  517, 6592,  ...,    1,    1,    1],
        [   2, 3255, 7728,  ...,    1,    1,    1],
        [   2,  517, 7265,  ...,    1,    1,    1]], dtype=torch.int32), tensor([15, 36, 14, 26, 47, 41, 16, 64, 13, 38, 17, 35, 29, 34, 34, 13, 45, 23,
        35, 27, 27, 10, 64, 17, 11, 42, 19,  9,  6, 36, 33, 25],
       dtype=torch.int32), tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]], dtype=torch.int32), tensor([0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1,
        0, 0, 1, 0, 1, 1, 1, 0], dtype=torch.int32)]
Waveform: tensor([[   2, 3093, 1698,  ...,    1,    1,    1],
        [   2,  517, 7989,  ...,    1,   

In [77]:
model_naver = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)

In [78]:
# Prepare optimizer and schedule (linear warmup and decay)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model_naver.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model_naver.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

In [79]:
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model_naver.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model_naver(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model_naver.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    model_naver.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model_naver(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

/tmp/ipykernel_13457/547134588.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):


  0%|          | 0/4688 [00:00<?, ?it/s]

epoch 1 batch id 1 loss 0.7251299619674683 train acc 0.5625
epoch 1 batch id 501 loss 0.5416437387466431 train acc 0.7422030938123753
epoch 1 batch id 1001 loss 0.37307876348495483 train acc 0.7865884115884116
epoch 1 batch id 1501 loss 0.38288414478302 train acc 0.8061292471685543
epoch 1 batch id 2001 loss 0.40183186531066895 train acc 0.8165136181909045
epoch 1 batch id 2501 loss 0.3074079155921936 train acc 0.8240953618552579
epoch 1 batch id 3001 loss 0.3809206187725067 train acc 0.8291819393535488
epoch 1 batch id 3501 loss 0.30578839778900146 train acc 0.8341456012567838
epoch 1 batch id 4001 loss 0.15479549765586853 train acc 0.8386106598350412
epoch 1 batch id 4501 loss 0.23662975430488586 train acc 0.8426321928460342
epoch 1 train acc 0.8440033063139932


/tmp/ipykernel_13457/547134588.py:22: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):


  0%|          | 0/1563 [00:00<?, ?it/s]

...

epoch 4 test acc 0.8942338451695457


  0%|          | 0/4688 [00:00<?, ?it/s]

epoch 5 batch id 1 loss 0.37341007590293884 train acc 0.90625
epoch 5 batch id 501 loss 0.010724205523729324 train acc 0.9717440119760479
epoch 5 batch id 1001 loss 0.11232448369264603 train acc 0.9731518481518482
epoch 5 batch id 1501 loss 0.01837874762713909 train acc 0.9735592938041305
epoch 5 batch id 2001 loss 0.023089228197932243 train acc 0.9745752123938031
epoch 5 batch id 2501 loss 0.01142159104347229 train acc 0.9755722710915634
epoch 5 batch id 3001 loss 0.007404250558465719 train acc 0.9755498167277574
epoch 5 batch id 3501 loss 0.13774728775024414 train acc 0.976105041416738
epoch 5 batch id 4001 loss 0.008246769197285175 train acc 0.9762325043739065
epoch 5 batch id 4501 loss 0.09745226800441742 train acc 0.9766510219951122
epoch 5 train acc 0.9769491254266212


  0%|          | 0/1563 [00:00<?, ?it/s]

epoch 5 test acc 0.8941138835572616


In [82]:
torch.save(model_naver.state_dict(), '../result/model/naver_news_e5_230303.pt')  # 전체 모델 저장

naver_model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)
naver_model.load_state_dict(torch.load('../result/model/naver_news_e5_230303.pt'))
naver_model.eval()

BERTClassifier(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(8002, 768, padding_idx=1)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True

### TEST with Unseen data

In [83]:
out_lst = []

for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm(unseen_dataloader)):
    token_ids = token_ids.long().to(device)
    segment_ids = segment_ids.long().to(device)
    valid_length= valid_length
    label = label.long().to(device)
    out = naver_model(token_ids, valid_length, segment_ids)
    out_lst.append(out.data.cpu())
    max_vals, max_indices = torch.max(out, 1)
    
pred = []
for batch in out_lst:
    for item in batch:
        pred.append(np.argmax(item.numpy()))
        
comments_unseen['pred'] = pred
print(comments_unseen.groupby('pred').size())
print((comments_unseen['sentiment'] == comments_unseen['pred']).mean())

100%|██████████| 53/53 [00:01<00:00, 26.64it/s]

pred
0    1139
1     528
dtype: int64
0.7720455908818237


# 직접 레이블한 댓글 데이터를 학습데이터로 사용

# 타겟으로 단어 바꾸기 이전 버젼

In [84]:
bertmodel, vocab = get_pytorch_kobert_model()
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

using cached model. /home/mglee/VSCODE/git_folder/comments_sentiment_label/code/.cache/kobert_v1.zip
using cached model. /home/mglee/VSCODE/git_folder/comments_sentiment_label/code/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece
using cached model. /home/mglee/VSCODE/git_folder/comments_sentiment_label/code/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [85]:
dataset_train = nlp.data.TSVDataset("../data/comments_train.tsv", field_indices=[1,4], num_discard_samples=1)
dataset_test = nlp.data.TSVDataset("../data/comments_test.tsv", field_indices=[1,4], num_discard_samples=1)
dataset_unseen = nlp.data.TSVDataset("../data/comments_unseen.tsv", field_indices=[0,1], num_discard_samples=1)

print(dataset_train[0], '\n', dataset_test[0], '\n', dataset_unseen[0])

['들어보니 노래로는 안되겠던데...', '0'] 
 ['가격 통제하면 공급은 줄어든다. 경제학원론 교과서에 있는 내용인데 뭐... 분양가 상한제는 밀은 뻔지르하지만 본질적으로가격통제 정책이지. 공급은 줄어들 수 밖에', '0'] 
 ['공산당이야 뭐야 중국처럼되고싶어?', '0']


In [86]:
data_train = BERTDataset(dataset_train, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok, max_len, True, False)
data_unseen = BERTDataset(dataset_unseen, 0, 1, tok, max_len, True, False)

train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)
unseen_dataloader = torch.utils.data.DataLoader(data_unseen, batch_size=batch_size, num_workers=5)

for data in train_dataloader:
  print("Data: ", data)
  print("Waveform: {}\nSample rate: {}\nLabels: {}".format(data[0], data[1], data[2]))
  break

Data:  [tensor([[   2, 1804, 6370,  ...,    1,    1,    1],
        [   2, 3257, 6005,  ...,    1,    1,    1],
        [   2,  837, 5666,  ...,    1,    1,    1],
        ...,
        [   2,  517, 6757,  ...,    1,    1,    1],
        [   2, 2781, 7216,  ...,    1,    1,    1],
        [   2, 3437, 5507,  ..., 7836, 1844,    3]], dtype=torch.int32), tensor([18, 34,  4, 13, 12, 30, 18, 15, 29, 11, 11, 60, 10, 44, 64, 32, 64, 13,
        18, 54, 11, 59, 25, 36, 16, 13,  5, 50, 25, 35, 18, 64],
       dtype=torch.int32), tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]], dtype=torch.int32), tensor([0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0,
        0, 0, 0, 0, 0, 1, 0, 0], dtype=torch.int32)]
Waveform: tensor([[   2, 1804, 6370,  ...,    1,    1,    1],
        [   2, 3257, 6005,  ...,    1,   

In [87]:
model_ourA = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)

no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model_ourA.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model_ourA.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)
scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

In [88]:
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model_ourA.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model_ourA(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model_ourA.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    model_ourA.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model_ourA(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

/tmp/ipykernel_13457/3713612706.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):


  0%|          | 0/41 [00:00<?, ?it/s]

epoch 1 batch id 1 loss 0.7966457605361938 train acc 0.375
epoch 1 train acc 0.7396341463414634


/tmp/ipykernel_13457/3713612706.py:22: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):


  0%|          | 0/7 [00:00<?, ?it/s]

...

epoch 4 test acc 0.9464285714285714


  0%|          | 0/41 [00:00<?, ?it/s]

epoch 5 batch id 1 loss 0.01393305417150259 train acc 1.0
epoch 5 train acc 0.9870426829268293


  0%|          | 0/7 [00:00<?, ?it/s]

epoch 5 test acc 0.9330357142857143


In [89]:
torch.save(model_ourA.state_dict(), '../result/model/ourA_news_e5_230303.pt')  # 전체 모델 저장

ourA_model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)
ourA_model.load_state_dict(torch.load('../result/model/ourA_news_e5_230303.pt'))
ourA_model.eval()

BERTClassifier(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(8002, 768, padding_idx=1)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True

### Our A TEST with Unseen

In [90]:
out_lst = []

for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm(unseen_dataloader)):
    token_ids = token_ids.long().to(device)
    segment_ids = segment_ids.long().to(device)
    valid_length= valid_length
    label = label.long().to(device)
    out = ourA_model(token_ids, valid_length, segment_ids)
    out_lst.append(out.data.cpu())
    max_vals, max_indices = torch.max(out, 1)
    
pred = []
for batch in out_lst:
    for item in batch:
        pred.append(np.argmax(item.numpy()))
comments_unseen['pred'] = pred
print(comments_unseen.groupby('pred').size())
print((comments_unseen['sentiment'] == comments_unseen['pred']).mean())

100%|██████████| 53/53 [00:01<00:00, 27.08it/s]

pred
0    1486
1     181
dtype: int64
0.7498500299940012


# 타겟으로 바꾼것으로 학습

In [91]:
dataset_train = nlp.data.TSVDataset("../data/comments_train.tsv", field_indices=[5,4], num_discard_samples=1) #5번이 타겟으로 바꾼 것
dataset_test = nlp.data.TSVDataset("../data/comments_test.tsv", field_indices=[5,4], num_discard_samples=1)
dataset_unseen = nlp.data.TSVDataset("../data/comments_unseen.tsv", field_indices=[2,1], num_discard_samples=1)

In [92]:
print(dataset_train[3], '\n', dataset_test[3], '\n', dataset_unseen[3])

['타겟이 일하는 병원 절대 안간다.', '0'] 
 ['타겟는 보호할필요가없다...가능하면 최대한의 불이익을받도록 해야한다 아...국짐같은넘들.', '0'] 
 ['저렇게 소독하면 지폐 사이사이는 더럽겠네 겉만 소독하능거잖아 ㅋ', '0']


In [93]:
bertmodel, vocab = get_pytorch_kobert_model()
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

using cached model. /home/mglee/VSCODE/git_folder/comments_sentiment_label/code/.cache/kobert_v1.zip
using cached model. /home/mglee/VSCODE/git_folder/comments_sentiment_label/code/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece
using cached model. /home/mglee/VSCODE/git_folder/comments_sentiment_label/code/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [94]:
data_train = BERTDataset(dataset_train, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok, max_len, True, False)
data_unseen = BERTDataset(dataset_unseen, 0, 1, tok, max_len, True, False)

train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)
unseen_dataloader = torch.utils.data.DataLoader(data_unseen, batch_size=batch_size, num_workers=5)

for data in train_dataloader:
  print("Data: ", data)
  print("Waveform: {}\nSample rate: {}\nLabels: {}".format(data[0], data[1], data[2]))
  break

Data:  [tensor([[   2, 1804, 6370,  ...,    1,    1,    1],
        [   2, 3257, 6005,  ...,    1,    1,    1],
        [   2,  837, 5666,  ...,    1,    1,    1],
        ...,
        [   2, 4698, 5404,  ...,    1,    1,    1],
        [   2, 2781, 7216,  ...,    1,    1,    1],
        [   2, 3437, 5507,  ..., 1844, 7139,    3]], dtype=torch.int32), tensor([18, 34,  4, 13, 11, 31, 18, 14, 25, 11, 10, 60,  9, 42, 64, 32, 64, 13,
        18, 54, 11, 59, 25, 36, 17, 13,  6, 50, 25, 32, 17, 64],
       dtype=torch.int32), tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]], dtype=torch.int32), tensor([0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0,
        0, 0, 0, 0, 0, 1, 0, 0], dtype=torch.int32)]
Waveform: tensor([[   2, 1804, 6370,  ...,    1,    1,    1],
        [   2, 3257, 6005,  ...,    1,   

In [95]:
model_ourB = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)

no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model_ourB.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model_ourB.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()


t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

In [96]:
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model_ourB.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model_ourB(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model_ourB.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    model_ourB.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model_ourB(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

/tmp/ipykernel_13457/1906873263.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):


  0%|          | 0/41 [00:00<?, ?it/s]

epoch 1 batch id 1 loss 0.701987624168396 train acc 0.46875
epoch 1 train acc 0.7682926829268293


/tmp/ipykernel_13457/1906873263.py:22: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):


  0%|          | 0/7 [00:00<?, ?it/s]

...

epoch 4 test acc 0.9285714285714286


  0%|          | 0/41 [00:00<?, ?it/s]

epoch 5 batch id 1 loss 0.08774244785308838 train acc 0.96875
epoch 5 train acc 0.9824695121951219


  0%|          | 0/7 [00:00<?, ?it/s]

epoch 5 test acc 0.9598214285714286


In [97]:
torch.save(model_ourB.state_dict(), '../result/model/ourB_news_e5_230303.pt')  # 전체 모델 저장

ourB_model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)
ourB_model.load_state_dict(torch.load('../result/model/ourB_news_e5_230303.pt'))
ourB_model.eval()

BERTClassifier(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(8002, 768, padding_idx=1)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True

### Our B TEST with Unseen

In [98]:
out_lst = []

for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm(unseen_dataloader)):
    token_ids = token_ids.long().to(device)
    segment_ids = segment_ids.long().to(device)
    valid_length= valid_length
    label = label.long().to(device)
    out = ourB_model(token_ids, valid_length, segment_ids)
    out_lst.append(out.data.cpu())
    max_vals, max_indices = torch.max(out, 1)
    
pred = []
for batch in out_lst:
    for item in batch:
        pred.append(np.argmax(item.numpy()))
        
comments_unseen['pred'] = pred
print("size : ", comments_unseen.groupby('pred').size())
print("acc : ", (comments_unseen['sentiment'] == comments_unseen['pred']).mean())

100%|██████████| 53/53 [00:01<00:00, 26.94it/s]

size :  pred
0    1399
1     268
dtype: int64
acc :  0.7876424715056989
